In [1]:
# ! pip install HOMarkov

In [12]:
import itertools

import numpy as np
import pandas as pd

In [13]:
data = pd.read_csv("paths_final_cass.csv", sep=";")

In [14]:
data.shape

(61, 8)

In [15]:
data.head()

,UID,GUID,element,exit_codes,result_id,start,end,type
0,00BEBCF7-833A-4869-9589-E67EBCF13CC5,00000000-0000-0005-5110-581486684385,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,2017-02-09 17:53:28,2017-02-09 17:54:13,0.0
1,038083DF-8D1E-4448-9FE4-D52C354CCF7F,00000000-0000-0008-7645-751490033592,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,2017-03-20 13:13:16,2017-03-20 13:14:21,2.0
2,07C33B4E-608D-4284-86E7-9651D3050023,00000000-0000-0008-4413-081489853134,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,2017-03-18 11:05:50,2017-03-18 11:06:47,2.0
3,0AAAF236-18F7-44A8-842A-CB8B06317CBA,00000000-0000-0005-5512-561483710181,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,2017-01-06 07:43:23,2017-01-06 07:44:25,2.0
4,142CAA57-6212-434C-A379-84D679BE9B00,00000000-0000-0004-9513-571485440659,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|1|...,2017-01-26 08:24:43,2017-01-26 08:25:58,2.0


In [16]:
all_codes = set(itertools.chain(*data['element'].apply(lambda x: str(x).split("|")).values))
codes_map = {item: n for n, item in enumerate(sorted(all_codes))}
data['codes'] = data['element'].apply(lambda x: "|".join(str(codes_map.get(i, 'N/A')) for i in str(x).split("|")))

all_codes = set(itertools.chain(*data['exit_codes'].apply(lambda x: str(x).split("|")).values))
_codes_map = {item: n for n, item in enumerate(sorted(all_codes))}
data['codes1'] = data['exit_codes'].apply(lambda x: "|".join(str(_codes_map.get(i, 'N/A')) for i in str(x).split("|")))

In [17]:
# data.head()

In [18]:
tot = list()
for row in data.itertuples():
    els = row.codes.split("|")
    tot.append(els)
    
maximum = np.array(list(itertools.chain(*tot))).astype(int).max()

In [19]:
maximum

287

In [20]:

def sequences(df):
    for row in df.itertuples():
        try:
            elements = np.array(row.codes.split("|")).astype(int)
            codes = np.array(row.codes1.split("|")).astype(int) + maximum
            yield list(itertools.chain(*zip(elements, codes)))
        except AttributeError:
            pass    

In [21]:
state_sequences = sequences(data)

In [22]:
tot_states = set()

for seq in state_sequences:
    for item in seq:
        tot_states.add(item)

In [23]:
number_of_states = len(tot_states)

In [24]:
print(f"{number_of_states} states found")

384 states found


In [25]:
import markov
import imp
imp.reload(markov)

<module 'markov' from '/Users/pmascolo/Desktop/Desktop/paths/markov.py'>

In [36]:
seqs = (list(map(int, str(row).split("|"))) for row in data.element)

tot_states_1 = set()

for seq in seqs:
    for item in seq:
        tot_states_1.add(item)
        
new_states = {i: n for n, i in enumerate(sorted(tot_states_1))}
seqs = (list(map(lambda x: new_states.get(x), map(int, str(row).split("|")))) for row in data.element)
print(sum( 1 for _ in  (list(map(lambda x: new_states.get(x), map(int, str(row).split("|")))) for row in data.element)))

61


In [37]:
len(tot_states_1)

288

In [38]:
mc1 =  markov.MarkovChain(len(tot_states_1), 2)

In [39]:
%%time
mc1.fit(seqs)

CPU times: user 118 ms, sys: 2.16 ms, total: 120 ms
Wall time: 119 ms


In [40]:
coords = list(zip(*mc1.transition_matrix.nonzero()))

In [41]:
for coord in coords:
    print(coord, mc1.transition_matrix[coord[0], coord[1]])

(200, 57752) 1.0
(300, 3712) 0.7
(300, 3473) 0.25
(300, 3709) 0.0333333333333
(300, 3472) 0.0166666666667
(541, 72915) 1.0
(619, 12473) 1.0
(805, 65995) 1.0
(1051, 54096) 1.0
(1327, 50403) 1.0
(1355, 58530) 1.0
(1681, 69491) 1.0
(1899, 49461) 1.0
(2194, 51365) 0.461538461538
(2194, 51340) 0.538461538462
(2378, 21363) 1.0
(2605, 3751) 1.0
(3087, 59780) 1.0
(3098, 62790) 1.0
(3142, 75663) 1.0
(3177, 2605) 1.0
(3334, 47973) 1.0
(3472, 4862) 1.0
(3473, 5150) 1.0
(3709, 72915) 1.0
(3712, 73774) 1.0
(3751, 2194) 1.0
(4183, 43718) 1.0
(4243, 60830) 1.0
(4375, 15848) 0.75
(4375, 16042) 0.25
(4862, 73199) 1.0
(5150, 73199) 1.0
(5363, 51703) 0.7
(5363, 51627) 0.2
(5363, 51747) 0.1
(5532, 17430) 1.0
(5656, 53018) 0.888888888889
(5656, 53259) 0.111111111111
(5675, 58530) 1.0
(5782, 6580) 1.0
(6071, 6644) 1.0
(6182, 38792) 0.4
(6182, 38728) 0.6
(6580, 70382) 1.0
(6644, 5782) 1.0
(7291, 26257) 1.0
(7582, 27249) 1.0
(7715, 65485) 1.0
(7836, 17430) 1.0
(7979, 58530) 1.0
(8349, 82355) 1.0
(8535, 52916)

In [42]:
asd = mc1.predict_state(mc1.transition_matrix[300, :], num_steps=100)

states = list(zip(zip(*asd.nonzero()), asd.data))
states

[((0, 4862), 0.016666666666666666),
 ((0, 72915), 0.033333333333333333),
 ((0, 5150), 0.25),
 ((0, 73774), 0.69999999999999996)]

In [43]:
for (_, col), prob in states:
    print(list(mc1.possible_states.keys())[col], prob)

(16, 254) 0.0166666666667
(253, 51) 0.0333333333333
(17, 254) 0.25
(256, 46) 0.7


In [45]:
%%time
mc1.predict_state(mc1.transition_matrix[5, :], num_steps=2).argmax()

CPU times: user 1.43 ms, sys: 489 µs, total: 1.91 ms
Wall time: 1.12 ms


0